In [1]:
%%capture --no-stderr
%pip install -r requirements.txt

In [2]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from bs4 import BeautifulSoup
import tiktoken
from collections import defaultdict
import base64
import os.path

- The file `credentials.json`: TODO describe where it is coming from
- The file `token.json`: created by running the cell below

In [7]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

In [5]:
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)
    messages = []
    page_token = None

    while True:
        # Get the list of messages
        results = service.users().messages().list(userId="me", q="is:unread", pageToken=page_token).execute()
        messages.extend(results.get("messages", []))
        page_token = results.get("nextPageToken")
        if not page_token:
            break

    if not messages:
        print("No unread messages found.")
    else:
        print(f"Number of unread messages: {len(messages)}")

except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")


Number of unread messages: 3517


In [6]:
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

    senders = defaultdict(int)
    for message in messages:
        msg = service.users().messages().get(userId="me", id=message['id'], format='metadata', metadataHeaders=['From']).execute()
        headers = msg['payload']['headers']
        for header in headers:
            if header['name'] == 'From':
                sender = header['value']
                senders[sender] += 1
    for sender, count in senders.items():
        print(f"Sender: {sender}, Count: {count}")

except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

Sender: "Asif (Marktechpost)" <marktechpost-newsletter@mail.beehiiv.com>, Count: 77
Sender: "Superhuman – Zain Kahn" <superhuman@mail.joinsuperhuman.ai>, Count: 187
Sender: Jeff Tan <thedailysnap@mail.beehiiv.com>, Count: 280
Sender: Bill Harris <birdphotographytips@mail.beehiiv.com>, Count: 162
Sender: "Juan at SpanishDictionary.com" <noreply@spanishdict.com>, Count: 253
Sender: "Daniel (Why Try AI)" <whytryai+10x-ai@substack.com>, Count: 16
Sender: Ashish Pratap Singh <algomaster@substack.com>, Count: 53
Sender: Katerina Lengold <katerinalengold@substack.com>, Count: 27
Sender: Stephen Gruppetta from The Python Coding Stack <thepythoncodingstack@substack.com>, Count: 35
Sender: Lazy Cat Kitchen  <newsletter@lazycatkitchen.com>, Count: 36
Sender: Nilda Chiaraviglio <contacto@nilda.com.mx>, Count: 86
Sender: Substack Reads <read@substack.com>, Count: 130
Sender: The AI Edge <theaiedge@substack.com>, Count: 97
Sender: Devansh from Artificial Intelligence Made Simple <artificialintellige

In [14]:
sum(senders.values())


1476

In [10]:
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

    # Search for the message with the specified title
    result = service.users().messages().list(userId="me", q='subject:"Mistral Breaks Barriers with New Multimodal AI"').execute()
    messages = result.get('messages', [])

    if not messages:
        print("No messages found with the specified title.")
    else:
        # Get the message details
        message_id = messages[0]['id']
        msg = service.users().messages().get(userId="me", id=message_id, format='full').execute()
        
        # Extract sender, title, date, and body
        headers = msg['payload']['headers']
        for header in headers:
            if header['name'] == 'From':
                sender = header['value']
            elif header['name'] == 'Subject':
                title = header['value']
            elif header['name'] == 'Date':
                date = header['value']
        
        # Extract the best body of the email
        body = None
        if 'parts' in msg['payload']:
            for part in msg['payload']['parts']:
                if part['mimeType'] == 'text/html':
                    body = part['body']['data']
                    break
                elif part['mimeType'] == 'text/plain' and body is None:
                    body = part['body']['data']
        else:
            body = msg['payload']['body']['data']

except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")


In [11]:
print(f"Sender: {sender}")
print(f"Title: {title}")
print(f"Date: {date}")
print(f"Body: {base64.urlsafe_b64decode(body).decode('utf-8')}")

Sender: The AI Edge <theaiedge@substack.com>
Title: Mistral Breaks Barriers with New Multimodal AI
Date: Tue, 17 Sep 2024 14:03:29 +0000
Body: <html><head><meta charSet="utf-8"><title>Mistral Breaks Barriers with New Multimodal AI</title><style>
@media all and (-ms-high-contrast: none), (-ms-high-contrast: active) {
  .typography .markup table.image-wrapper img,
  .typography.editor .markup table.image-wrapper img,
  .typography .markup table.kindle-wrapper img,
  .typography.editor .markup table.kindle-wrapper img {
    max-width: 550px;
  }
}


@media screen and (max-width: 650px) {
  .typography .markup div.youtube-overlay,
  .typography.editor .markup div.youtube-overlay,
  .typography .markup div.vimeo-overlay,
  .typography.editor .markup div.vimeo-overlay {
    display: none !important;
  }
}


@media screen and (max-width: 370px) {
  .typography .markup div.tiktok-wrap,
  .typography.editor .markup div.tiktok-wrap {
    width: calc(95vw - 32px);
    height: calc((95vw - 32px - 

In [15]:
# Decode the base64 encoded body
decoded_body = base64.urlsafe_b64decode(body).decode('utf-8')

# Parse the HTML content
soup = BeautifulSoup(decoded_body, 'html.parser')

# Remove the style tags and their contents
for style in soup.find_all('style'):
    style.decompose()

# Remove all attributes from HTML tags except for href attribute in <a> tags
for tag in soup.find_all(True):
    tag.attrs = {}
    # if tag.name == 'a':
    #     tag.attrs = {key: value for key, value in tag.attrs.items() if key == 'href'}
    # else:
    #     tag.attrs = {}

# Extract the content inside the <body> tag
body_content = soup.body #.prettify()

print(body_content)

<body><img/><div>Plus: OpenAI debuts o1 models, Google AI studio launches comparison mode, Luma drops Dream Machine API, and more.</div><div>͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏     ­͏    

In [ ]:
import base64

# Decode the base64 encoded body
decoded_body = base64.urlsafe_b64decode(body).decode('utf-8')

print(decoded_body)

In [20]:
enc = tiktoken.encoding_for_model("gpt-4o")
len(enc.encode(str(body_content)))

5194